In [ ]:
%%time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, 
                                stop_words=['english', 'spanish'], strip_accents='unicode', lowercase=True, ngram_range=[1,3])
dtm_tf = tf_vectorizer.fit_transform(kiva_df['story'])
print(dtm_tf.shape)

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, 
                                stop_words=['english', 'spanish'], strip_accents='unicode', lowercase=True, ngram_range=[1,3])
dtm_tfidf = tfidf_vectorizer.fit_transform(kiva_df['story'])
print(dtm_tfidf.shape)

%%time
from sklearn.decomposition import NMF, LatentDirichletAllocation
lda_tfidf = LatentDirichletAllocation(n_components=20, max_iter=10, learning_method='online', learning_offset=50.,random_state=0).fit(dtm_tfidf)
#nmf = NMF(n_components=20, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(" Topic {:d}: ".format(topic_idx+1), end = '')
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 6
print("LDA Topics:")
display_topics(lda_tfidf, tfidf_vectorizer.get_feature_names(), no_top_words)

#print("NMF Topics:")
#display_topics(nmf, tf_feature_names, no_top_words)

import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, mds="tsne")